In [2]:
# ================================================================
#     KAIST 高相似类数据分析脚本（自动忽略第一列时间列）
# ================================================================

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap

# ===================== 1. 路径 =====================
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/mixed_high_similar.csv"
save_root = "/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/high_similar_data_analysis/result"
os.makedirs(save_root, exist_ok=True)

# ===================== 2. 读取数据 =====================
df = pd.read_csv(csv_path)
print("📌 数据量:", len(df))
print("📌 原始列:", df.columns.tolist())

# ---------- 自动忽略第一列 ----------
cols = df.columns.tolist()
time_col = cols[0]               # 第一列（时间序列）
feature_cols = cols[1:]          # 从第二列开始都是特征（包括 label）
print(f"⏱ 已忽略时间列：{time_col}")

# ---------- 提取 label ----------
label_col = "label"
assert label_col in df.columns, "❌ CSV 中找不到 label 列"

# ---------- 剔除 label 后的特征 ----------
feature_cols = [c for c in feature_cols if c != label_col]

X = df[feature_cols].values
y = df[label_col].values

# ===================== 3. 类别样本统计 =====================
label_counts = df[label_col].value_counts().sort_index()
print("\n=== 类别样本量 ===\n", label_counts)

label_counts.to_csv(os.path.join(save_root, "label_counts.csv"))

plt.figure(figsize=(8,5))
sns.barplot(x=label_counts.index, y=label_counts.values)
plt.xlabel("Label")
plt.ylabel("Count")
plt.title("Samples per Class")
plt.savefig(os.path.join(save_root, "class_count.png"), dpi=300)
plt.close()

# ===================== 4. 每类统计量 =====================
stats = df.groupby(label_col)[feature_cols].agg(["mean", "std", "min", "max"])
stats.to_csv(os.path.join(save_root, "class_feature_stats.csv"))
print("📌 特征统计已保存")

# ===================== 5. 类间相似性（均值向量余弦相似度） =====================
labels = sorted(df[label_col].unique())
sim_matrix = pd.DataFrame(index=labels, columns=labels)

print("\n=== 类间均值向量余弦相似度 ===")
for i in labels:
    Xi = df[df[label_col] == i][feature_cols].values.mean(axis=0)
    for j in labels:
        Xj = df[df[label_col] == j][feature_cols].values.mean(axis=0)

        cos_sim = np.dot(Xi, Xj) / (np.linalg.norm(Xi)*np.linalg.norm(Xj) + 1e-8)
        sim_matrix.loc[i, j] = cos_sim

sim_matrix.to_csv(os.path.join(save_root, "class_similarity_cosine.csv"))
print(sim_matrix)

plt.figure(figsize=(6,5))
sns.heatmap(sim_matrix.astype(float), annot=True, cmap="coolwarm")
plt.title("Cosine Similarity Between Classes")
plt.savefig(os.path.join(save_root, "class_similarity_heatmap.png"), dpi=300)
plt.close()

# ===================== 6. UMAP 降维 =====================
print("\n🔥 正在执行 UMAP 降维 …")

reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
emb = reducer.fit_transform(X)

umap_df = pd.DataFrame({"x": emb[:,0], "y": emb[:,1], "label": y})
umap_df.to_csv(os.path.join(save_root, "umap_embedding.csv"), index=False)

plt.figure(figsize=(7,6))
sns.scatterplot(data=umap_df, x="x", y="y", hue="label", palette="tab10", s=8)
plt.title("UMAP Embedding of High-Similar Classes")
plt.savefig(os.path.join(save_root, "umap.png"), dpi=300)
plt.close()

print("🎉 所有分析结果已保存到：", save_root)


📌 数据量: 7500
📌 原始列: ['timestamp', 'vibration_ch1', 'vibration_ch2', 'vibration_ch3', 'vibration_ch4', 'label']
⏱ 已忽略时间列：timestamp

=== 类别样本量 ===
 label
1     4000
11     500
12    1000
13    1000
14    1000
Name: count, dtype: int64
📌 特征统计已保存

=== 类间均值向量余弦相似度 ===
          1         11        12        13        14
1   0.999979 -0.420982  0.813597 -0.662863 -0.055472
11 -0.420982  0.999996 -0.268449  0.722053 -0.030217
12  0.813597 -0.268449  0.999994 -0.716718  0.146328
13 -0.662863  0.722053 -0.716718  0.999982  0.276137
14 -0.055472 -0.030217  0.146328  0.276137  0.999995

🔥 正在执行 UMAP 降维 …


/home/charles/miniconda3/envs/d2l/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


🎉 所有分析结果已保存到： /home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/high_similar_data_analysis/result
